In [33]:
import cv2
import mediapipe as mp
import os
import tensorflow as tf
import numpy as np

model_rec = tf.keras.models.load_model('modelX')

In [34]:
LABELS = {0: 'l', 1: 'n', 2: 'r', 3: 'a', 4: 'e', 5: 'g', 6: 'i'}

In [37]:
# fonction d'identification du signe de la main grâce au modèle de classification
def identify_crop(crop):
    try :
        resized = (cv2.resize(crop,(60,60)).astype('float') / 255).reshape(1,60,60,3)
        recognized = np.argmax(model_rec.predict(resized)) 
        return recognized
    except : pass
    

In [38]:
#fonction servant à trouver les limites du rectangle d'encadrement de la main
def draw_rectangle(hand_landmarks, image):
    image_height, image_width, _ = image.shape
    keypoints = []
    for data_point in hand_landmarks.landmark:
        keypoints.append({
                            'X': data_point.x*image_width,
                            'Y': data_point.y*image_height,                            
                            })
    
    X_min = image_width
    X_max = 0
    Y_min = image_height
    Y_max = 0

    for x in range(len(keypoints)):
        if (keypoints[x]['X']) < X_min:
            X_min = (keypoints[x]['X'])
        if (keypoints[x]['X']) > X_max:
            X_max = (keypoints[x]['X'])

    for y in range(len(keypoints)):
        if (keypoints[y]['Y']) < Y_min:
            Y_min = (keypoints[y]['Y'])
        if (keypoints[y]['Y']) > Y_max:
            Y_max = (keypoints[y]['Y'])

    startX = int(X_min -0.05* image_width)
    startY = int(Y_min -0.05* image_height)
    endX = int(X_max +0.05* image_width)
    endY = int(Y_max +0.05* image_height)
    
    crop = image[startY:endY, startX:endX]
    recognized = identify_crop(crop)
    if recognized :
        image = cv2.rectangle(image, (startX, startY), (endX, endY), (155, 255, 0), 2)
        image = cv2.putText(image, LABELS[recognized], (10,300), cv2.FONT_HERSHEY_SIMPLEX, 2, (155, 255, 0), 2)
    else :
        image = cv2.rectangle(image, (startX, startY), (endX, endY), (122, 10, 122), 2)
    return(image)


In [42]:
def detection_video():    
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils

    cap = cv2.VideoCapture(0)
    with mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                break

            image = cv2.cvtColor(cv2.flip(image,1), cv2.COLOR_BGR2RGB)

            image.flags.writeable = False
            results = hands.process(image) # détection de la main grâce à mediapipe
            image.flags.writeable = True

            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    image =  draw_rectangle(hand_landmarks, image)
            cv2.imshow('Sign Language Detection', image)
            if cv2.waitKey(5) & 0xFF == 27:
                break
    cap.release()
    cv2.destroyAllWindows()

In [43]:
detection_video()